In [1]:
using CSV, DataFrames, Distributions,StatsBase,PythonPlot,Random,CausalityTools,Tables

In [2]:
sim_dir = "./../output/sims"
files = readdir(sim_dir)
#τ_range = [parse(Int,split(split(f,"_")[end],".")[1]) for f in files]

τ_range = unique([parse(Int,split(split(f,"tau=")[2],"-")[1]) for f in files]);
ϕ_range = unique([parse(Float64,split(split(f,"phi=")[2],".csv")[1]) for f in files]);


In [ ]:
sims_raw = []
sims_cells = []
for τ in τ_range
    for ϕ in ϕ_range
        # get raw dataframe
        println(sim_dir*"/sim_data_tau="*string(τ)*"-phi="*string(ϕ)*".csv")
        df = CSV.read(sim_dir*"/sim_data_tau="*string(τ)*"-phi="*string(ϕ)*".csv",DataFrame)
        df.ϕ = ones(length(df.time)) .*ϕ
        df.τ = ones(length(df.time)) .*τ
        push!(sims_raw,df)

        # make cell data frame
        df_cells = combine(groupby(df,[:position]),
            :λ=>mean=>:λ,
            :M=>(x->x[1])=>:M0,
            :M=>(x->x[end])=>:Mf,
            :time=>(x->x[end]-x[1])=>:gt,
            :ϕ=>(x->x[1])=>:ϕ,
            :τ=>(x->x[1])=>:τ)
        df_cells[:,:x] = log.(df_cells.M0 ./ mean(df_cells.M0))
        df_cells[:,:γ] = log.(df_cells.Mf ./ df_cells.M0)
        df_cells[:,:φ] = df_cells.λ .* df_cells.gt

        push!(sims_cells,df_cells[df_cells.gt .> 0,:])
    end
end
sims_raw = vcat(sims_raw...);
sims_cells = vcat(sims_cells...);

In [ ]:
fig, axs= subplots(figsize=(10,4),ncols = length(τ_range),nrows = length(ϕ_range),sharex=true,sharey=true)
for i in eachindex(τ_range)
    for j in eachindex(ϕ_range)
        d = sims_raw[(sims_raw.ϕ .== ϕ_range[j]) .& (sims_raw.τ .== τ_range[i]),:]
        for p in unique(d.position)[1:4]
            axs[j-1,i-1].plot(d[d.position .==p,:].time,d[d.position .==p,:].λ,"k",lw=0.4)
        end
    end
end
fig

In [ ]:
fig, ax= subplots(figsize=(2,2),sharex=true,sharey=true)
for i in eachindex(τ_range)

    for j in eachindex(ϕ_range)
        d = sims_raw[(sims_raw.ϕ .== ϕ_range[j]) .& (sims_raw.τ .== τ_range[i]),:]  
        #ax.plot([ϕ_range[j]],[var(d.λ)],"C"*string(i)*"o")
        positions = unique(d.position)
        #ax.plot([ϕ_range[j]],[var(d.λ)/mean(d.λ)^2],"C"*string(i)*"+")
        #ax.plot([τ_range[j]],[mean([var(d[d.position .== p,:].λ) for p in positions])],"C"*string(i)*"+")
        ax.plot([τ_range[j]],[var([mean(d[d.position .== p,:].λ) for p in positions])],"C"*string(i)*"s")
    end
    #
    #ax.set_ylim([0,0.000150])

end
ax.set_xlabel(L"$\phi$")
fig

In [ ]:
fig, ax = subplots(figsize=(3,3))
for i in 1:length(τ_range)
    vgr = [var(df.gr) for df in sims_cells[3][i]]
    vgt = [var(df.gr) for df in sims_cells[3][i]]
    ax.plot(ϕ_range,vgr,"-",label="τ="*string(τ_range[i]))
    
end
ax.legend()
#ax.set_ylim([0,0.00005])
#ax.set_xlim([0,0.00005])
fig